In [36]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


In [37]:
simulated_buses = pd.read_csv( '../data/simulated/simulated_buses.csv' )

In [38]:
# simulated_buses[simulated_buses.manual_timediff < 1000].to_csv('../data/simulated/simulated_buses_multi_slow.csv', index=False)

In [39]:
next_bus = simulated_buses[simulated_buses['bus_id'] == '44ce47fe-2d47-4515-b24e-2f9840be2600']
next_bus = next_bus.reset_index(drop=True)
next_bus

,timestamp,route_id,direction_id,latitude,longitude,bus_id,elapsed_seconds,manual_timediff
0,2025-01-12 09:00:00.000000,292250,1,49.267365,-123.247213,44ce47fe-2d47-4515-b24e-2f9840be2600,0,0
1,2025-01-12 09:02:01.602272,292250,1,49.267227,-123.246398,44ce47fe-2d47-4515-b24e-2f9840be2600,122,121
2,2025-01-12 09:03:40.089806,292250,1,49.266323,-123.245606,44ce47fe-2d47-4515-b24e-2f9840be2600,220,220
3,2025-01-12 09:04:10.089806,292250,1,49.264836,-123.244344,44ce47fe-2d47-4515-b24e-2f9840be2600,250,250
4,2025-01-12 09:05:58.933536,292250,1,49.262616,-123.242463,44ce47fe-2d47-4515-b24e-2f9840be2600,359,358
...,...,...,...,...,...,...,...,...
115,2025-01-12 12:30:36.087063,292250,1,49.262253,-123.003230,44ce47fe-2d47-4515-b24e-2f9840be2600,12636,12636
116,2025-01-12 12:32:17.502601,292250,1,49.264432,-123.003150,44ce47fe-2d47-4515-b24e-2f9840be2600,12738,12737
117,2025-01-12 12:34:28.238443,292250,1,49.264630,-123.006011,44ce47fe-2d47-4515-b24e-2f9840be2600,12868,12868
118,2025-01-12 12:36:22.384583,292250,1,49.266157,-123.006010,44ce47fe-2d47-4515-b24e-2f9840be2600,12982,12982


# Slow Simulation




In [40]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

def generate_interpolated_path(
    start_lat: float,
    start_lon: float,
    end_lat: float,
    end_lon: float,
    start_time: datetime,
    total_seconds: int = 30
) -> pd.DataFrame:
    """
    Generate interpolated points between start and end locations with full-second intervals.
    
    Parameters:
    start_lat (float): Starting latitude
    start_lon (float): Starting longitude
    end_lat (float): Ending latitude
    end_lon (float): Ending longitude
    start_time (datetime): Starting timestamp
    total_seconds (int): Total number of seconds for the journey
    
    Returns:
    pd.DataFrame: DataFrame with columns [timestamp, lat, lon, elapsed_seconds]
    """
    # Initialize data collection
    data = []
    current_time = start_time
    
    # Generate points for each second
    num_points = total_seconds + 1  # Include both start and end points
    latitudes = np.linspace(start_lat, end_lat, num_points)
    longitudes = np.linspace(start_lon, end_lon, num_points)
    
    # Create points for each second
    for i in range(num_points):
        next_timestamp = current_time + timedelta(seconds=i)
        data.append({
            'timestamp': next_timestamp,
            'lat': latitudes[i],
            'lon': longitudes[i],
            'elapsed_seconds': i
        })
    
    # Add 5 minutes of dwelling at the last point
    last_lat = latitudes[-1]
    last_lon = longitudes[-1]
    dwell_seconds = 396  # 5 minutes = 300 seconds
    
    for i in range(dwell_seconds):
        next_timestamp = current_time + timedelta(seconds=total_seconds + i)
        data.append({
            'timestamp': next_timestamp,
            'lat': last_lat,
            'lon': last_lon,
            'elapsed_seconds': total_seconds + i
        })
    
    # Convert to DataFrame
    return pd.DataFrame(data)


In [41]:
start_lat, start_lon = 49.25809218714809, -123.24353651684615
end_lat, end_lon = 49.25583960281676, -123.23680567009362
start_time = datetime(2025, 1, 12, 9, 0, 0)


# Generate interpolated path
df_interpolated = generate_interpolated_path(
    start_lat,
    start_lon,
    end_lat,
    end_lon,
    start_time,
    total_seconds = 60 * 15
)
df_interpolated['manual_timediff'] = df_interpolated['elapsed_seconds']

# df_interpolated

In [42]:
after_seconds = 60 * 15

In [43]:
next_bus_after_600 = next_bus[next_bus['elapsed_seconds'] > after_seconds][['timestamp','latitude','longitude','elapsed_seconds','manual_timediff']].rename(columns={'latitude':'lat', 'longitude':'lon'})

In [44]:
next_bus_after_600

,timestamp,lat,lon,elapsed_seconds,manual_timediff
10,2025-01-12 09:15:57.879901,49.255666,-123.236672,958,957
11,2025-01-12 09:17:38.823479,49.255526,-123.236475,1059,1058
12,2025-01-12 09:18:08.823479,49.255544,-123.236191,1089,1088
13,2025-01-12 09:18:38.823479,49.255698,-123.236031,1119,1118
14,2025-01-12 09:19:08.823479,49.257058,-123.232451,1149,1148
...,...,...,...,...,...
115,2025-01-12 12:30:36.087063,49.262253,-123.003230,12636,12636
116,2025-01-12 12:32:17.502601,49.264432,-123.003150,12738,12737
117,2025-01-12 12:34:28.238443,49.264630,-123.006011,12868,12868
118,2025-01-12 12:36:22.384583,49.266157,-123.006010,12982,12982


In [45]:
df_interpolated

,timestamp,lat,lon,elapsed_seconds,manual_timediff
0,2025-01-12 09:00:00,49.258092,-123.243537,0,0
1,2025-01-12 09:00:01,49.258090,-123.243529,1,1
2,2025-01-12 09:00:02,49.258087,-123.243522,2,2
3,2025-01-12 09:00:03,49.258085,-123.243514,3,3
4,2025-01-12 09:00:04,49.258082,-123.243507,4,4
...,...,...,...,...,...
1292,2025-01-12 09:21:31,49.255840,-123.236806,1291,1291
1293,2025-01-12 09:21:32,49.255840,-123.236806,1292,1292
1294,2025-01-12 09:21:33,49.255840,-123.236806,1293,1293
1295,2025-01-12 09:21:34,49.255840,-123.236806,1294,1294


In [46]:
df_interpolated_with_bus = pd.concat([df_interpolated, next_bus_after_600])
df_interpolated_with_bus.to_csv('../data/simulated/person_1_with_bus.csv', index=False)

In [47]:
df_interpolated_with_bus

,timestamp,lat,lon,elapsed_seconds,manual_timediff
0,2025-01-12 09:00:00,49.258092,-123.243537,0,0
1,2025-01-12 09:00:01,49.258090,-123.243529,1,1
2,2025-01-12 09:00:02,49.258087,-123.243522,2,2
3,2025-01-12 09:00:03,49.258085,-123.243514,3,3
4,2025-01-12 09:00:04,49.258082,-123.243507,4,4
...,...,...,...,...,...
115,2025-01-12 12:30:36.087063,49.262253,-123.003230,12636,12636
116,2025-01-12 12:32:17.502601,49.264432,-123.003150,12738,12737
117,2025-01-12 12:34:28.238443,49.264630,-123.006011,12868,12868
118,2025-01-12 12:36:22.384583,49.266157,-123.006010,12982,12982


In [48]:
# df_interpolated.to_csv('../data/simulated/person_1.csv', index=False)

In [49]:
from typing import Dict, List, Any
import json 
def shapes_to_animated_geojson(df: pd.DataFrame) -> Dict[str, Any]:
    """
    Convert GPS movement data into animated GeoJSON format for Kepler.gl.
    
    Args:
        df: DataFrame with columns timestamp, lat, lon, elapsed_seconds
    
    Returns:
        Dictionary in GeoJSON format with timestamps for animation
    """
    # Convert string timestamps to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # Convert timestamps to Unix time (seconds since epoch)
    df['unix_time'] = df['timestamp'].astype('int64') // 10**9
    
    # Create coordinates list with [longitude, latitude, altitude, timestamp]
    coordinates = df.apply(
        lambda row: [
            float(row["lon"]), 
            float(row["lat"]), 
            0,  # altitude (set to 0)
            int(row["unix_time"])  # timestamp in seconds
        ], 
        axis=1
    ).tolist()
    
    # Create GeoJSON structure
    geojson = {
        "type": "FeatureCollection",
        "features": [{
            "type": "Feature",
            "properties": {
                "id": "person_1"  # You can customize this identifier
            },
            "geometry": {
                "type": "LineString",
                "coordinates": coordinates
            }
        }]
    }
    
    return geojson

class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, pd.Timestamp):
            return obj.isoformat()
        if isinstance(obj, (np.int64, np.int32)):
            return int(obj)
        if isinstance(obj, (np.float64, np.float32)):
            return float(obj)
        return super().default(obj)


def save_animated_geojson(df: pd.DataFrame, output_file: str) -> None:
    """
    Convert DataFrame to animated GeoJSON and save to file.
    
    Args:
        df: Input DataFrame with GPS movement data
        output_file: Path to save the GeoJSON file
    """
    geojson_data = shapes_to_animated_geojson(df)
    
    with open(output_file, 'w') as f:
        json.dump(geojson_data, f, cls=CustomEncoder, indent=2)

In [50]:
save_animated_geojson(df_interpolated_with_bus, '../data/simulated/person_1_with_bus.geojson')